# HYPERPARAMETER OPTIMIZATION USING OPTUNA

## IMPORTS

In [1]:
##################
# IMPORT MODULES #
##################
# SYS IMPORT
import pandas as pd
import numpy as np

from tqdm import tqdm
import matplotlib.pyplot as plt

import xgboost as xgb
import lightgbm as lgb 
import optuna 
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score, mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

## CONFIG

In [2]:
config = {
    "DATA_PATH" : "D:/Documents/GitHub/ml-pipeline/data/TPS-AUG2021/train.csv",
    "TARGET_VAR" : "loss"
}

## LOADING DATA

In [3]:
df = pd.read_csv(config["DATA_PATH"])
target = df[config["TARGET_VAR"]]
df.head()

,id,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f91,f92,f93,f94,f95,f96,f97,f98,f99,loss
0,0,-0.002350,59,0.766739,-1.350460,42.2727,16.68570,30.3599,1.267300,0.392007,...,-42.43990,26.854000,1.45751,0.696161,0.941764,1.828470,0.924090,2.29658,10.48980,15
1,1,0.784462,145,-0.463845,-0.530421,27324.9000,3.47545,160.4980,0.828007,3.735860,...,-184.13200,7.901370,1.70644,-0.494699,-2.058300,0.819184,0.439152,2.36470,1.14383,3
2,2,0.317816,19,-0.432571,-0.382644,1383.2600,19.71290,31.1026,-0.515354,34.430800,...,7.43721,37.218100,3.25339,0.337934,0.615037,2.216760,0.745268,1.69679,12.30550,6
3,3,0.210753,17,-0.616454,0.946362,-119.2530,4.08235,185.2570,1.383310,-47.521400,...,9.66778,0.626942,1.49425,0.517513,-10.222100,2.627310,0.617270,1.45645,10.02880,2
4,4,0.439671,20,0.968126,-0.092546,74.3020,12.30650,72.1860,-0.233964,24.399100,...,290.65700,15.604300,1.73557,-0.476668,1.390190,2.195740,0.826987,1.78485,7.07197,1


## FEATURE ENGINEERING

In [4]:
def feature_engineering(dataframe):
    features = dataframe.columns[1:101]
    return dataframe, features

In [5]:
df, features = feature_engineering(df)

## OPTIMIZING

In [8]:
def objective(trial, data=df[features], target=target):
    
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.2,random_state=95)
    param = {
        'objective' : "reg:squarederror",
        'seed': 95,
        'n_estimators': 4000,
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 10.0),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.006,0.008,0.01,0.014,0.017,0.02]),
        'max_depth': trial.suggest_categorical('max_depth', [5,10,15,20]),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
        'n_jobs': 2,
        'tree_method': "gpu_hist",
        "gpu_id": 0,
        'predictor': 'gpu_predictor'
    }
    model = xgb.XGBRegressor(**param)  
    
    model.fit(train_x,train_y,eval_set=[(test_x,test_y)], early_stopping_rounds=200, verbose=False)
    
    preds = model.predict(test_x)
    
    rmse = mean_squared_error(test_y, preds, squared=False)
    
    return rmse

In [9]:
%%time
study = optuna.create_study(direction='minimize')
study.optimize(objective, timeout=3600*5)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2021-08-03 14:12:53,098] A new study created in memory with name: no-name-a4b014ef-3462-445e-9928-2ccd531ccc26
[I 2021-08-03 14:14:07,496] Trial 0 finished with value: 7.805481248181306 and parameters: {'reg_alpha': 3.730683102562492, 'colsample_bytree': 0.5, 'subsample': 0.5, 'learning_rate': 0.017, 'max_depth': 15, 'min_child_weight': 105}. Best is trial 0 with value: 7.805481248181306.
[I 2021-08-03 14:15:26,199] Trial 1 finished with value: 7.802911285028155 and parameters: {'reg_alpha': 0.004429371660341314, 'colsample_bytree': 0.5, 'subsample': 0.7, 'learning_rate': 0.02, 'max_depth': 15, 'min_child_weight': 149}. Best is trial 1 with value: 7.802911285028155.
[I 2021-08-03 14:17:17,299] Trial 2 finished with value: 7.791425125430866 and parameters: {'reg_alpha': 0.0441067199361869, 'colsample_bytree': 0.3, 'subsample': 0.4, 'learning_rate': 0.008, 'max_depth': 10, 'min_child_weight': 115}. Best is trial 2 with value: 7.791425125430866.
[I 2021-08-03 14:17:43,343] Trial 3 fini

## VISUALIZATION

In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
#Visualize parameter importances.
optuna.visualization.plot_param_importances(study)

In [ ]:
params=study.best_params
params

{'reg_alpha': 0.0031293275223408185,
 'reg_lambda': 0.04787145507141445,
 'colsample_bytree': 0.3,
 'subsample': 0.6,
 'learning_rate': 0.008,
 'max_depth': 100,
 'num_leaves': 584,
 'min_child_samples': 173,
 'min_data_per_groups': 30}

In [ ]:
{'reg_alpha': 0.007915504076304212, 'colsample_bytree': 0.5, 'subsample': 0.8, 'learning_rate': 0.008, 'max_depth': 10, 'min_child_weight': 274}